In [60]:
#Importing libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

#Import tensorflow/keras libraries

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Conv1D, MaxPooling1D,GlobalAveragePooling1D
from keras.utils import np_utils

In [2]:
#If using google colab
#Upload data from local drive


#from google.colab import files
#uploaded = files.upload()

Saving data.csv to data.csv


In [3]:
#Load data

eegdatafile = 'data.csv' 

data = pd.read_csv(eegdatafile, header=0)

In [4]:
data.head(10)

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5
5,X14.V1.56,55,28,18,16,16,19,25,40,52,...,-12,-31,-42,-54,-60,-64,-60,-56,-55,5
6,X3.V1.191,-55,-9,52,111,135,129,103,72,37,...,-125,-99,-79,-62,-41,-26,11,67,128,4
7,X11.V1.273,1,-2,-8,-11,-12,-17,-15,-16,-18,...,-79,-91,-97,-88,-76,-72,-66,-57,-39,2
8,X19.V1.874,-278,-246,-215,-191,-177,-167,-157,-139,-118,...,-400,-379,-336,-281,-226,-174,-125,-79,-40,1
9,X3.V1.491,8,15,13,3,-6,-8,-5,4,25,...,49,31,11,-5,-17,-19,-15,-15,-11,4


In [5]:
#In the column y, replace values greater than 1 with 0 to make it a binary classification
#1 - epileptic seizure, 0- control (which replaces 2,3,4,5 - subjects with no epileptic seizure)

data.loc[data["y"] > 1, "y"] = 0

In [6]:
data.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11500 entries, 0 to 11499
Columns: 180 entries, Unnamed: 0 to y
dtypes: int64(179), object(1)
memory usage: 15.8+ MB


**Preprocessing**

In [8]:
#Drop the first column

data = data.drop("Unnamed: 0", axis=1)

In [51]:
#Generate the Independent variables

X = data.values
X= X[:,0:-1]
X.shape

(11500, 178)

In [43]:
#Generate the Dependent variable

Y= data['y'].values
Y=np_utils.to_categorical(Y)
Y.shape

(11500, 2)

In [52]:
#Normalize the data

X = (X-X.mean())/X.std()
X.shape

(11500, 178)

In [53]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [54]:
#Splitting dataset into train, validation and test sets at 80%, 10% and 10% respectively.


#Split data into train and remaining dataset
X_train, X_re, Y_train, Y_re = train_test_split(X, Y, train_size=0.80,stratify=Y,random_state=1)

#Split remaining data into test and validation with 50% split 
X_val, X_test, Y_val, Y_test = train_test_split(X_re, Y_re, test_size=0.5, stratify=Y_re,random_state=1)



In [55]:
X_train = X_train.reshape(-1,178,1)
X_test = X_test.reshape(-1,178,1)
X_val = X_val.reshape(-1,178,1)

X_train.shape
X_test.shape

(1150, 178, 1)

Deep Learning Models


In [56]:
#LSTM network

model = Sequential()
model.add(LSTM(56, input_shape=(178,1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(56))
model.add(Dropout(0.5))
model.add(Dense(20))
model.add(Activation('tanh'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 178, 56)           12992     
                                                                 
 dropout_14 (Dropout)        (None, 178, 56)           0         
                                                                 
 lstm_7 (LSTM)               (None, 56)                25312     
                                                                 
 dropout_15 (Dropout)        (None, 56)                0         
                                                                 
 dense_10 (Dense)            (None, 20)                1140      
                                                                 
 activation_6 (Activation)   (None, 20)                0         
                                                                 
 dense_11 (Dense)            (None, 2)                

In [57]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [58]:
model.fit(X_train, Y_train,validation_data=(X_val, Y_val)
                ,epochs = 10, batch_size = 20
                )

Epoch 1/10
460/460 [==============================] - 72s 147ms/step - loss: 0.1642 - accuracy: 0.9520 - val_loss: 0.1031 - val_accuracy: 0.9652
Epoch 2/10
460/460 [==============================] - 66s 144ms/step - loss: 0.1445 - accuracy: 0.9572 - val_loss: 0.1285 - val_accuracy: 0.9522
Epoch 3/10
460/460 [==============================] - 65s 141ms/step - loss: 0.1166 - accuracy: 0.9614 - val_loss: 0.1189 - val_accuracy: 0.9600
Epoch 4/10
460/460 [==============================] - 65s 142ms/step - loss: 0.1232 - accuracy: 0.9597 - val_loss: 0.1086 - val_accuracy: 0.9696
Epoch 5/10
460/460 [==============================] - 66s 143ms/step - loss: 0.1206 - accuracy: 0.9632 - val_loss: 0.1091 - val_accuracy: 0.9583
Epoch 6/10
460/460 [==============================] - 97s 212ms/step - loss: 0.1217 - accuracy: 0.9626 - val_loss: 0.1070 - val_accuracy: 0.9652
Epoch 7/10
460/460 [==============================] - 74s 161ms/step - loss: 0.1026 - accuracy: 0.9671 - val_loss: 0.1038 - val_ac

In [69]:
#Evaluate metrics
train_acc = model.evaluate(X_train, Y_train,verbose=1)
val_acc =  model.evaluate(X_val,Y_val, verbose=1)

36/36 [==============================] - 2s 45ms/step - loss: 0.1035 - accuracy: 0.9670


In [31]:
#Predict the model on test data

predictions = model.predict (X_test)


In [68]:

test_loss, test_acc = model.evaluate(X_test,Y_test, verbose=1)
print("Test Accuracy: "+"{:.2%}".format(test_acc))

36/36 [==============================] - 2s 46ms/step - loss: 0.1100 - accuracy: 0.9661
Test Accuracy: 96.61%


In [63]:
#1D Convolution Neural Network

cnn = Sequential()
cnn.add(Conv1D(32, kernel_size= 5,activation='relu', input_shape=(178,1)))
cnn.add(Dropout(0.5))
cnn.add(MaxPooling1D(pool_size= 2))
cnn.add(Conv1D(64,kernel_size= 5, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(MaxPooling1D(pool_size= 2))
cnn.add(Conv1D(128, kernel_size= 5, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(MaxPooling1D(pool_size= 2))
cnn.add(GlobalAveragePooling1D())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(2, activation='softmax'))
cnn.summary()

cnn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 174, 32)           192       
                                                                 
 dropout_22 (Dropout)        (None, 174, 32)           0         
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 87, 32)           0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 83, 64)            10304     
                                                                 
 dropout_23 (Dropout)        (None, 83, 64)            0         
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 41, 64)           0         
 1D)                                                  

In [64]:
cnn.fit(X_train, Y_train,validation_data=(X_val, Y_val)
                ,epochs = 10, batch_size = 20
                )

Epoch 1/10
460/460 [==============================] - 11s 21ms/step - loss: 0.1818 - accuracy: 0.9262 - val_loss: 0.1358 - val_accuracy: 0.9513
Epoch 2/10
460/460 [==============================] - 9s 20ms/step - loss: 0.0943 - accuracy: 0.9627 - val_loss: 0.0975 - val_accuracy: 0.9696
Epoch 3/10
460/460 [==============================] - 9s 20ms/step - loss: 0.0672 - accuracy: 0.9776 - val_loss: 0.1010 - val_accuracy: 0.9600
Epoch 4/10
460/460 [==============================] - 9s 20ms/step - loss: 0.0609 - accuracy: 0.9808 - val_loss: 0.0766 - val_accuracy: 0.9739
Epoch 5/10
460/460 [==============================] - 10s 21ms/step - loss: 0.0543 - accuracy: 0.9813 - val_loss: 0.0683 - val_accuracy: 0.9765
Epoch 6/10
460/460 [==============================] - 10s 22ms/step - loss: 0.0521 - accuracy: 0.9820 - val_loss: 0.0757 - val_accuracy: 0.9696
Epoch 7/10
460/460 [==============================] - 11s 24ms/step - loss: 0.0529 - accuracy: 0.9833 - val_loss: 0.0767 - val_accuracy: 0.

In [65]:
#Predict the model on test data

predictions = cnn.predict (X_test)

In [66]:
test_loss, test_acc = cnn.evaluate(X_test,Y_test, verbose=1)
print("Test Accuracy: "+"{:.2%}".format(test_acc))

36/36 [==============================] - 1s 16ms/step - loss: 0.0655 - accuracy: 0.9739
Test Accuracy: 97.39%
